In [13]:
import numpy as np
import pandas as pd
import requests
from bs4 import BeautifulSoup
import lxml.html as lh



In [4]:
URL = "https://projects.fivethirtyeight.com/2021-mlb-predictions/games/"
page = requests.get(URL)

soup = BeautifulSoup(page.content, "html.parser")

print(soup)

<!DOCTYPE html>
<html lang="en"><head><meta content="2021-08-26T16:16:10-04:00" property="article:modified_time"/><title>2021 MLB Predictions | FiveThirtyEight</title><meta content="2021 MLB Predictions" property="og:title"/><meta content="FiveThirtyEight's MLB forecast uses a pitcher-adjusted Elo model to project the winner of every game and the chances that each team will win the World Series." property="og:description"/><meta content="FiveThirtyEight's MLB forecast uses a pitcher-adjusted Elo model to project the winner of every game and the chances that each team will win the World Series." name="description"/><link href="https://projects.fivethirtyeight.com/2021-mlb-predictions/" rel="canonical"/><meta content="https://projects.fivethirtyeight.com/2021-mlb-predictions/" property="og:url"/><meta content="https://fivethirtyeight.com/wp-content/uploads/2017/03/promo_mlb2017_16x9.png" property="og:image"/><meta content="https://fivethirtyeight.com/wp-content/uploads/2017/03/promo_mlb2

In [23]:
#Store the contents of the website under doc
doc = lh.fromstring(page.content)

#Parse data that are stored between <tr>..</tr> of HTML
tr_elements = doc.xpath('//tr')

[len(T) for T in tr_elements[:11]]


[9, 9, 8, 9, 8, 9, 8, 9, 8, 9, 8]

In [26]:
len(tr_elements)

402

In [69]:
upcoming_list_done = False
future_list_done = False

rows = [row.text_content() for row in tr_elements]

header_str = "DateTeamStarting pitcherTeam ratingStarting pitcher adj.Travel, rest & home field adj.Pregame team ratingWin prob.Chance of winningScore"

rows.index(header_str)

indexes = [i for i,row in enumerate(rows) if row == header_str]

## This gives us the two header strings, telling us that row 0 is a header
## and 1-200 are the body rows for the first table
## 201 is a header, and 202-end are the body rows for the 2nd table
indexes
            
            
                
                


[0, 201]